![](../images/rivacon_frontmark_combined_header.png)

# Example notebook for simulation of portfolio of interest rate swaps 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import random
import pyvacon.analytics as analytics
import pyvacon.tools.converter as converter
import pyvacon.tools.enums as enums
import pyvacon.models.tools as modeltools

import datetime


In [ ]:
refdate_p = datetime.datetime(2017,2,20)
refdate = converter.getLTime(refdate_p)

### Create the CIR model which we want to use to price the swap portfolio

In [ ]:
cir = analytics.CIRModel('CIR_Test',refdate, 1.0,0.05,0.20,0.025)

### Setup portfolio of interest rate swaps
In our example we are using swaps with one fixed leg and one floating leg (with just 10 payments payment and fixing dates).
The swap needs to get vectors with start, end and payment dates (floater additionally need a reset date vector). The i-th accrual period is defined by the i-th entry of start and end vectors.
Note that this kind of swap has only floating legs with floating rate period equal to accrual period.The fixed rate is drawn from a uniform distribution (between 0.1% and 5%).

In [ ]:
# create the vectors defining the statdates, enddates, paydates and reset dates
dates_tmp = [refdate_p + datetime.timedelta(days=2)]
for i in range(10):
    dates_tmp.append(dates_tmp[-1] + datetime.timedelta(days=(i+1)*365))

startdates = converter.createPTimeList(refdate, dates_tmp)

enddates_p = []
for i in range(len(dates_tmp)):
    enddates_p.append(dates_tmp[i] + datetime.timedelta(days=(i+1)*365))
enddates = converter.createPTimeList(refdate, enddates_p)

paydates_p = []
for i in range(len(enddates_p)):
    paydates_p.append(enddates_p[i] + datetime.timedelta(days=2))
paydates = converter.createPTimeList(refdate, paydates_p)

resetdates = startdates
# we need a vector of notionals since if needed, each accrual period may have a different notional
# However, if the vectir is of length 1 the same notional is applied to all periods
notionals = analytics.vectorDouble()
notionals.append(100.0)

#we create a portfolio with swaps differing by fixed rate (random number). The quantity of each swap is also a random number
spread = 0.01
floatleg = analytics.IrFloatLegSpecification(notionals, resetdates, startdates, enddates,
                                       paydates,'EUR', 'test_udl', enums.DayCounter.ACT365_FIXED, 
                                       spread)
num_swaps = 100
swaps = []
quantities = []
for i in range(num_swaps):
    fixed_rate = random.uniform(0.0001, 0.05)
    fixedleg = analytics.IrFixedLegSpecification(fixed_rate, notionals, startdates, enddates, 
                                       paydates,'EUR', enums.DayCounter.ACT365_FIXED)
    ir_swap = analytics.InterestRateSwapSpecification('TEST_SWAP', 'DBK', enums.SecuritizationLevel.COLLATERALIZED, 'EUR',
                                           converter.getLTime(paydates[-1]), fixedleg, floatleg)
    swaps.append(ir_swap)
    quantities.append(100)


### Simulate the CIR Model
First we simulate the CIR model which can be done by using the ModelLab. The ModelLab stores internally all simulated values (here the short rate) and provides functionality to obtain these values.

In [ ]:
cir_lab = analytics.ModelLab(cir, refdate)
sim_times_p = [refdate_p + datetime.timedelta(days=180)]
for i in range(5):
    sim_times_p.append(sim_times_p[-1] + datetime.timedelta(days=(i+1)*180))
sim_times = converter.createPTimeList(refdate, sim_times_p)
n_sims = 100
max_num_threads = 1
cir_lab.simulate(sim_times, n_sims, 200, max_num_threads)

### Compute portfolio values for simulated paths
We now compute the portfolio values for the simulated paths.

In [ ]:
def compute_portfolio_value(pay_pricing_data, rec_pricing_data, swaps, quantities, cir, refdate_p):
    portfolio_value = 0
    dc = modeltools.compute_yieldcurve(cir, refdate_p)
    pay_pricing_data.discountCurve = dc
    rec_pricing_data.discountCurve = dc
    rec_pricing_data.fixingCurve = dc
    for j in range(len(swaps)):
        pay_pricing_data.spec = swaps[j].getPayLeg()
        rec_pricing_data.spec = swaps[j].getReceiveLeg()
        pr = analytics.price(ir_swap_pricing_data)
        portfolio_value += quantities[j] * pr.getPrice()
    return portfolio_value

ir_swap_pricing_data = analytics.InterestRateSwapPricingData()
    
pay_leg_pricing_data = analytics.InterestRateSwapLegPricingData()
pay_leg_pricing_data.fxRate = 1.0
pay_leg_pricing_data.weight = -1.0
    
rec_leg_pricing_data = analytics.InterestRateSwapFloatLegPricingData()
rec_leg_pricing_data.fxRate = 1.0
rec_leg_pricing_data.weight = 1.0
    
ir_swap_pricing_data.pricer = 'InterestRateSwapPricer'
ir_swap_pricing_data.pricingRequest = analytics.PricingRequest()
ir_swap_pricing_data.valDate = refdate
ir_swap_pricing_data.setCurr('EUR')
ir_swap_pricing_data.addLegData(pay_leg_pricing_data)
ir_swap_pricing_data.addLegData(rec_leg_pricing_data)

ir_swap_pricing_data.param = analytics.InterestRateSwapPricingParameter()
ir_swap_pricing_data.param.fixingGracePeriod = analytics.time_duration.time_duration_infinite()

initial_value = compute_portfolio_value(pay_leg_pricing_data, rec_leg_pricing_data, swaps, quantities, cir, refdate_p)
simulation = []
for sim_time_point in range(len(sim_times_p)):
    sim_time_p = sim_times_p[sim_time_point]
    sim_time = converter.getLTime(sim_time_p)
    ir_swap_pricing_data.valDate = sim_time
    ptf_values = []
    for i in range(n_sims):
        cir_lab.setFromSimulatedValues(cir, sim_time_point, i)        
        portfolio_value = compute_portfolio_value(pay_leg_pricing_data, rec_leg_pricing_data, swaps, quantities, cir, sim_time_p)
        ptf_values.append(portfolio_value)
    simulation.append(ptf_values)

#### Plot histogram of portfolio values

In [ ]:
n_bins = 50
time_point = 0
n, bins, patches = plt.hist(simulation[time_point], n_bins, normed=False, facecolor='blue', alpha=0.75)